In [1]:
# carregar bibliotecas
source('fun/setup.R')

# https://bookdown.org/steve_midway/DAR/random-effects.html ***
# https://rcompanion.org/handbook/G_03.html
# https://stackoverflow.com/questions/40711912/how-to-add-a-random-effect-to-a-linear-mixed-effects-model
library('lmerTest')
# library('lme4')
library('MuMIn') # require(remotes); install_version("MuMIn", version = "1.46.0", repos = "http://cran.us.r-project.org") - https://stackoverflow.com/questions/17082341/installing-older-version-of-r-package
# library('versions')

GDAL version >= 3.1.0 | setting mapviewOptions(fgb = TRUE)

Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:
# Estrutura de pastas
pasta_dados        <- "../../yellow_dados"
pasta_modelos      <- sprintf('%s/06_bases_para_modelo', pasta_dados)
pasta_base_modelo  <- sprintf('%s/C_base_para_modelo', pasta_modelos)
# dir.create(pasta_base_modelo, recursive = TRUE, showWarnings = FALSE)


# Abrir arquivos processados
# base_modelo <- sprintf('%s/yellow_base_para_modelo.csv', pasta_base_modelo)
base_modelo <- sprintf('%s/yellow_base_para_modelo_3ptos_50vgs.csv', pasta_base_modelo)
base_modelo <- read_delim(base_modelo, delim = ';', col_types = 'ccccdddddccdiccdccdddccccccccccccc')

head(base_modelo)

trip_id,osm_id,qgis_id,linha_sent,speed_kph,vel_med_gps,vel_med_m3,vel_med_m5,elev_grad_rev,elev_sent,cat_grad,dist_total,dist_trecho_quadra,cat_dist_trecho,cat_dist_total,curv_h,cat_curv,inicio_fim,dens_lotes_100m,dens_lotes_100m_15m,dens_lotes_100m_30m,dia_util,cat_fx_hora,class_via,infra_ciclo,via_restr,semaforos,osm_highway,osm_oneway,contramao,vg_loop,vg_exper,vg_contramao,vg_parques
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
000135_00,396771678,271157,des_linha,1.984096,2.581532,3.234793,4.050268,1.1397530,subida,plano,821.4461,21,curto,curta,0,00 a 25 graus,início,2.318142,3.017319,2.880641,util,pico_tarde,arterial,sem_infra_ciclo,via_comum,sem_semaforos,residential,no,não,não,não,não,não
000135_00,396771678,271157,opo_linha,7.143187,3.502971,4.055528,4.356572,-1.1397530,descida,plano,821.4461,129,acima_media,curta,0,00 a 25 graus,meio,2.318142,3.017319,2.880641,util,pico_tarde,arterial,sem_infra_ciclo,via_comum,sem_semaforos,residential,no,não,não,não,não,não
000135_00,150545097,211446,des_linha,6.000300,4.660937,4.739349,4.751611,0.4597172,subida,plano,821.4461,100,medio,curta,0,00 a 25 graus,meio,3.004687,5.458913,0.000000,util,pico_tarde,arterial,sem_infra_ciclo,via_comum,fim,primary,yes,não,não,não,não,não
000135_00,150545097,211447,des_linha,5.027570,5.038803,5.076647,5.079040,1.0969643,subida,plano,821.4461,426,grande,curta,0,00 a 25 graus,meio,1.174983,1.429278,1.503516,util,pico_tarde,arterial,sem_infra_ciclo,via_comum,inicio,primary,yes,não,não,não,não,não
000135_00,150545097,104168,des_linha,4.340691,4.625234,4.411842,4.070812,1.1705242,subida,plano,821.4461,108,medio,curta,0,00 a 25 graus,final,3.503065,2.938014,2.846124,util,pico_tarde,arterial,sem_infra_ciclo,via_comum,sem_semaforos,primary,yes,não,não,não,não,não
000138_00,185783795,161928,des_linha,3.853800,4.585846,6.600380,7.055670,0.2440453,subida,plano,1585.3999,21,curto,media,0,00 a 25 graus,meio,0.000000,0.000000,0.000000,util,pico_tarde,arterial,ciclovia,via_comum,fim,trunk,yes,não,não,não,não,não


In [3]:
# ------------------------------------------------------------------------------
# Seleção das variáveis e transformação em factor/dummies
# ------------------------------------------------------------------------------

# base_modelo <- base_modelo %>% mutate(dist_total_km = dist_total/1000)

# # Transformar colunas de interesse em factors
# base_modelo <- 
#   base_modelo %>%
#   mutate(
#     inicio_fim = factor(inicio_fim, levels = c('meio', 'início', 'final')),
#     cat_dist_total = factor(cat_dist_total, levels = c('media', 'acima_media', 'longa', 'muito_longa', 'curta')),
#     cat_dist_trecho = factor(cat_dist_trecho, levels = c('medio', 'acima_media', 'grande', 'muito_grande', 'curto')),
#     cat_grad   = factor(cat_grad,   levels = c('plano', 'desc_lev', 'desc_med', 'desc_for', 'desc_ver', 'subi_lev', 'subi_med', 'subi_for', 'subi_ver')),
#     # cat_grad_prev   = factor(cat_grad_prev,   levels = c('plano', 'desc_lev', 'desc_med', 'desc_for', 'desc_ver', 'subi_lev', 'subi_med', 'subi_for', 'subi_ver'))
#     class_via    = factor(class_via,   levels = c('local', 'coletora', 'arterial', 'vtr', 'ciclo_expressa', 'ped_serv')),
#     # class_viaosm = factor(class_via,   levels = c('local', 'coletora', 'arterial', 'vtr', 'ciclo_expressa', 'ped_serv')),
#     infra_ciclo  = factor(infra_ciclo, levels = c('sem_infra_ciclo', 'ciclofaixa', 'ciclovia')),
#     semaforos    = factor(semaforos, levels = c('sem_semaforos', 'inicio', 'fim', 'inicio_fim')),
#     via_restr    = factor(via_restr,   levels = c('via_comum', 'via_restrita')),
#     osm_oneway   = factor(osm_oneway, levels = c('yes', 'no')),
#     contramao    = factor(contramao, levels = c('não', 'sim')),
#     dia_util     = factor(dia_util,    levels = c('util', 'desc')),
#     cat_fx_hora  = factor(cat_fx_hora, levels = c('vale', 'pico_manha', 'pico_tarde', 'madrugada')),
#     cat_vg_loop  = factor(vg_loop, levels = c('não', 'sim')),
#     cat_vg_exper = factor(vg_exper, levels = c('não', 'sim')),
#     cat_vg_conm  = factor(vg_contramao, levels = c('não', 'sim')),
#     cat_vg_parq  = factor(vg_parques, levels = c('não', 'sim')),
#     cat_curv     = factor(cat_curv, levels = c('00 a 25 graus', '25 a 45 graus', '45 a 90 graus', 'acima de 90 graus')),
#   )

# Transformar colunas de interesse em factors - categorização na ordem, para graphhopper
base_modelo <- 
  base_modelo %>%
  mutate(
    inicio_fim = factor(inicio_fim, levels = c('meio', 'início', 'final')),
    cat_grad   = factor(cat_grad,   levels = c('subi_ver', 'subi_for', 'subi_med',  'subi_lev', 'plano', 'desc_lev', 'desc_med', 'desc_for', 'desc_ver')),
    class_via    = factor(class_via,   levels = c('ped_serv', 'local', 'coletora', 'arterial', 'ciclo_expressa', 'vtr')),
    infra_ciclo  = factor(infra_ciclo, levels = c('sem_infra_ciclo', 'ciclovia', 'ciclofaixa')),
    semaforos    = factor(semaforos, levels = c('fim', 'inicio_fim', 'inicio', 'sem_semaforos')),
    via_restr    = factor(via_restr,   levels = c('via_comum', 'via_restrita')),
    osm_oneway   = factor(osm_oneway, levels = c('no', 'yes')),
    contramao    = factor(contramao, levels = c('não', 'sim')),
    cat_dist_total  = factor(cat_dist_total, levels = c('media', 'acima_media', 'longa', 'muito_longa', 'curta')),
    cat_dist_trecho = factor(cat_dist_trecho, levels = c('medio', 'acima_media', 'grande', 'muito_grande', 'curto')),
    dia_util     = factor(dia_util,    levels = c('util', 'desc')),
    cat_curv     = factor(cat_curv, levels = c('00 a 25 graus', '25 a 45 graus', '45 a 90 graus', 'acima de 90 graus')),
    cat_fx_hora  = factor(cat_fx_hora, levels = c('pico_manha', 'vale', 'pico_tarde', 'madrugada')),
    cat_vg_loop  = factor(vg_loop, levels = c('não', 'sim')),
    cat_vg_exper = factor(vg_exper, levels = c('não', 'sim')),
    cat_vg_conm  = factor(vg_contramao, levels = c('não', 'sim')),
    cat_vg_parq  = factor(vg_parques, levels = c('não', 'sim')),
    
  )

# Quantos NAs temos na base e onde?
# colSums(is.na(base_modelo))

mutate: converted 'cat_grad' from character to factor (0 new NA)

        converted 'cat_dist_trecho' from character to factor (0 new NA)

        converted 'cat_dist_total' from character to factor (0 new NA)

        converted 'cat_curv' from character to factor (0 new NA)

        converted 'inicio_fim' from character to factor (0 new NA)

        converted 'dia_util' from character to factor (0 new NA)

        converted 'cat_fx_hora' from character to factor (0 new NA)

        converted 'class_via' from character to factor (0 new NA)

        converted 'infra_ciclo' from character to factor (0 new NA)

        converted 'via_restr' from character to factor (0 new NA)

        converted 'semaforos' from character to factor (0 new NA)

        converted 'osm_oneway' from character to factor (0 new NA)

        converted 'contramao' from character to factor (0 new NA)

        new variable 'cat_vg_loop' (factor) with 2 unique values and 0% NA

        new variable 'cat_vg_exper' (fa

In [4]:
# ------------------------------------------------------------------------------
# Segundo modelo: OLS com regressão simples e random effects
# ------------------------------------------------------------------------------

base_modelo <- base_modelo %>% select(vel_med_m5,
                                      # log_speed ~
                                      # sqrt_speed ~
                                      inicio_fim, 
                                      # elev_grad_rev,
                                      # I(elev_grad_rev^2),
                                      # I(elev_grad_rev^3),
                                      cat_grad, 
                                      class_via,
                                      infra_ciclo,
                                      semaforos,
                                      via_restr,
                                      osm_oneway,
                                      contramao,
                                      # dist_total_km,
                                      cat_dist_total,
                                      # dist_trecho_quadra,
                                      cat_dist_trecho,
                                      # curv_h,
                                      cat_curv,
                                      dens_lotes_100m,
                                      # dens_lotes_100m_15m,
                                      # dens_lotes_100m_30m,
                                      dia_util,
                                      cat_fx_hora,
                                      cat_vg_loop,
                                      cat_vg_exper,
                                      cat_vg_conm,
                                      cat_vg_parq,
                                      qgis_id,
                                      trip_id)


head(base_modelo)

select: dropped 17 variables (osm_id, linha_sent, speed_kph, vel_med_gps, vel_med_m3, …)



vel_med_m5,inicio_fim,cat_grad,class_via,infra_ciclo,semaforos,via_restr,osm_oneway,contramao,cat_dist_total,cat_dist_trecho,cat_curv,dens_lotes_100m,dia_util,cat_fx_hora,cat_vg_loop,cat_vg_exper,cat_vg_conm,cat_vg_parq,qgis_id,trip_id
<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
4.050268,início,plano,arterial,sem_infra_ciclo,sem_semaforos,via_comum,no,não,curta,curto,00 a 25 graus,2.318142,util,pico_tarde,não,não,não,não,271157,000135_00
4.356572,meio,plano,arterial,sem_infra_ciclo,sem_semaforos,via_comum,no,não,curta,acima_media,00 a 25 graus,2.318142,util,pico_tarde,não,não,não,não,271157,000135_00
4.751611,meio,plano,arterial,sem_infra_ciclo,fim,via_comum,yes,não,curta,medio,00 a 25 graus,3.004687,util,pico_tarde,não,não,não,não,211446,000135_00
5.079040,meio,plano,arterial,sem_infra_ciclo,inicio,via_comum,yes,não,curta,grande,00 a 25 graus,1.174983,util,pico_tarde,não,não,não,não,211447,000135_00
4.070812,final,plano,arterial,sem_infra_ciclo,sem_semaforos,via_comum,yes,não,curta,medio,00 a 25 graus,3.503065,util,pico_tarde,não,não,não,não,104168,000135_00
7.055670,meio,plano,arterial,ciclovia,fim,via_comum,yes,não,media,curto,00 a 25 graus,0.000000,util,pico_tarde,não,não,não,não,161928,000138_00


In [5]:
# # Checar ordem dos resultados, se serão só somatórias
# ols_resultados <- lm(vel_med_m5 ~
#                        # log_speed ~
#                        # sqrt_speed ~
#                        inicio_fim + 
#                        # elev_grad_rev +
#                        # I(elev_grad_rev^2) +
#                        # I(elev_grad_rev^3) +
#                        cat_grad + 
#                        class_via +
#                        infra_ciclo +
#                        semaforos +
#                        via_restr +
#                        osm_oneway +
#                        contramao +
#                        # dist_total_km +
#                        cat_dist_total +
#                        # dist_trecho_quadra +
#                        cat_dist_trecho +
#                        # curv_h +
#                        cat_curv +
#                        dens_lotes_100m +
#                        # dens_lotes_100m_15m +
#                        # dens_lotes_100m_30m +
#                        dia_util +
#                        cat_fx_hora +
#                        cat_vg_loop +
#                        cat_vg_exper +
#                        cat_vg_conm +
#                        cat_vg_parq
#                      ,
#                      data = base_modelo)

# summary(ols_resultados)

# gc(T)

In [6]:
(start = Sys.time())

[1] "2023-05-21 16:36:53 -03"

In [7]:
ols_resultados <- lmer(vel_med_m5 ~
                         # log_speed ~
                         # sqrt_speed ~
                         inicio_fim + 
                         # elev_grad_rev +
                         # I(elev_grad_rev^2) +
                         # I(elev_grad_rev^3) +
                         cat_grad + 
                         class_via +
                         infra_ciclo +
                         semaforos +
                         via_restr +
                         osm_oneway +
                         contramao +
                         # dist_total_km +
                         cat_dist_total +
                         # dist_trecho_quadra +
                         cat_dist_trecho +
                         # curv_h +
                         cat_curv +
                         dens_lotes_100m +
                         # dens_lotes_100m_15m +
                         # dens_lotes_100m_30m +
                         dia_util +
                         cat_fx_hora +
                         cat_vg_loop +
                         cat_vg_exper +
                         cat_vg_conm +
                         cat_vg_parq +
                         (1 | qgis_id) +
                         (1 | trip_id)
                       ,
                       data = base_modelo)

summary(ols_resultados)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00753216 (tol = 0.002, component 1)”

Correlation matrix not shown by default, as p = 43 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: vel_med_m5 ~ inicio_fim + cat_grad + class_via + infra_ciclo +  
    semaforos + via_restr + osm_oneway + contramao + cat_dist_total +  
    cat_dist_trecho + cat_curv + dens_lotes_100m + dia_util +  
    cat_fx_hora + cat_vg_loop + cat_vg_exper + cat_vg_conm +  
    cat_vg_parq + (1 | qgis_id) + (1 | trip_id)
   Data: base_modelo

REML criterion at convergence: 10214724

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4197 -0.5842 -0.0148  0.5538 15.3416 

Random effects:
 Groups   Name        Variance Std.Dev.
 trip_id  (Intercept)  3.571   1.890   
 qgis_id  (Intercept)  1.280   1.131   
 Residual             10.927   3.306   
Number of obs: 1903888, groups:  trip_id, 177796; qgis_id, 6704

Fixed effects:
                                  Estimate     Std. Error             df
(Intercept)                       7.252981       0.293969    8721.311919
inicio_fiminício          

In [8]:
MuMIn::r.squaredGLMM(ols_resultados)

# https://stats.stackexchange.com/questions/232465/how-to-compare-models-on-the-basis-of-aic
nobs(ols_resultados)
BIC(ols_resultados)
logLik(ols_resultados)
AIC(ols_resultados)

Warning message:
“'r.squaredGLMM' now calculates a revised statistic. See the help page.”


R2m,R2c
0.1972249,0.4440182


[1] 1903888

[1] 10215389

'log Lik.' -5107362 (df=46)

[1] 10214816

In [9]:
Sys.time()

[1] "2023-05-21 17:02:27 -03"

In [10]:
Sys.time() - start

Time difference of 25.56091 mins